In [167]:
import pandas as pd
import numpy as np
from scipy.stats import pearsonr, spearmanr
import sklearn as scikit #scikit learn module
from sklearn.manifold import TSNE
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_selection import r_regression
from time import time #because it's stochastic?

# Set the maximum number of rows in a DF displayed to 40
pd.set_option('display.max_rows', 40)

In [168]:
#1.0 Load in relevant data
    #testing with random dataset at first[:, [7] + list(range(10, df.shape[1])]
data_wmeta = pd.read_csv('21_treatment_data.csv')
data = data_wmeta.drop(data_wmeta.columns[0], axis=1)
meta = pd.read_csv('index_array.csv')
treatment_names = data_wmeta.iloc[:, 0]

In [169]:
# 2.0 Initialize model, random seed set
tsne = TSNE(n_components=2, perplexity=50, n_iter=4000, random_state=42)
    #fit tSNE to data
red_data = tsne.fit_transform(data) # red_data is 'reduced data' or the lower dimensional form of the OG dataset
red_data_df = pd.DataFrame(red_data, columns=['D1', 'D2'])
treatment_names_df = pd.DataFrame(treatment_names)

concatenated_df = pd.concat([treatment_names_df, red_data_df], axis=1)
# Set the 'Treatment' column as the index
concatenated_df.set_index('Treatment', inplace=True)

In [171]:
concatenated_df

,D1,D2
Treatment,,
Treatment 1,0.898451,-2.867285
Treatment 1,2.470459,-4.060156
Treatment 1,1.508968,-0.949219
Treatment 1,1.334150,0.234819
Treatment 1,1.636961,-2.733887
...,...,...
Treatment 19,-7.962851,-5.325697
Treatment 20,-1.491370,2.825283
Treatment 20,-1.564736,2.824116


In [172]:
# Assuming 'concatenated_df' is your DataFrame
# Assuming 'chosen_point_index' is the index of the chosen input point

# Extract the chosen input point
chosen_point = concatenated_df.iloc[234]

#Initialize empty DFs
pearson_corr = []
spearman_corr = []
avg_correlations = [] #average of all 3 

column_names = ['Treatment', 'D1', 'D2']
columns_of_interest = ['D1', 'D2']


# Calculate the cosine correlations
cosine_corr = pd.DataFrame(cosine_similarity(concatenated_df), index=concatenated_df.index, columns=concatenated_df.index)

In [173]:
cosine_corr

Treatment,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 2,Treatment 2,Treatment 2,Treatment 2,Treatment 2,Treatment 2,Treatment 2,Treatment 2,Treatment 2,Treatment 2,Treatment 3,Treatment 3,Treatment 3,Treatment 3,Treatment 3,Treatment 3,Treatment 3,Treatment 3,Treatment 3,Treatment 3,Treatment 4,Treatment 4,Treatment 4,Treatment 4,Treatment 4,Treatment 4,Treatment 4,Treatment 4,Treatment 4,Treatment 4,Treatment 5,Treatment 5,Treatment 5,Treatment 5,Treatment 5,Treatment 5,Treatment 5,Treatment 5,Treatment 5,Treatment 6,Treatment 6,Treatment 6,Treatment 6,Treatment 6,Treatment 6,Treatment 6,Treatment 6,Treatment 6,Treatment 7,Treatment 7,Treatment 7,Treatment 7,Treatment 7,Treatment 7,Treatment 7,Treatment 7,Treatment 7,Treatment 8,Treatment 8,Treatment 8,Treatment 8,Treatment 8,Treatment 8,Treatment 8,Treatment 8,Treatment 8,Treatment 9,Treatment 9,Treatment 9,Treatment 9,Treatment 9,Treatment 9,Treatment 9,Treatment 9,Treatment 9,Treatment 10,Treatment 10,Treatment 11,Treatment 11,Treatment 12,Treatment 12,Treatment 13,Treatment 13,Treatment 14,Treatment 14,Treatment 15,Treatment 15,Treatment 16,Treatment 16,Treatment 17,Treatment 17,Treatment 18,Treatment 18,Treatment 19,Treatment 19,Treatment 20,Treatment 20,Treatment 21,Treatment 21
Treatment,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Treatment 1,1.000000,0.970629,0.761201,0.129072,0.972314,0.951262,0.954899,0.990283,0.909544,0.907850,-0.325367,-0.562057,0.991695,0.885509,0.999801,0.996566,0.

In [174]:
# Function to rank the values within each row
def rank_row(row):
    return row.rank(ascending=False, method='min').astype(int)

# Apply the rank function to each row
cosine_rank = cosine_corr.apply(rank_row, axis=1)
cosine_rank

Treatment,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 1,Treatment 2,Treatment 2,Treatment 2,Treatment 2,Treatment 2,Treatment 2,Treatment 2,Treatment 2,Treatment 2,Treatment 2,Treatment 3,Treatment 3,Treatment 3,Treatment 3,Treatment 3,Treatment 3,Treatment 3,Treatment 3,Treatment 3,Treatment 3,Treatment 4,Treatment 4,Treatment 4,Treatment 4,Treatment 4,Treatment 4,Treatment 4,Treatment 4,Treatment 4,Treatment 4,Treatment 5,Treatment 5,Treatment 5,Treatment 5,Treatment 5,Treatment 5,Treatment 5,Treatment 5,Treatment 5,Treatment 6,Treatment 6,Treatment 6,Treatment 6,Treatment 6,Treatment 6,Treatment 6,Treatment 6,Treatment 6,Treatment 7,Treatment 7,Treatment 7,Treatment 7,Treatment 7,Treatment 7,Treatment 7,Treatment 7,Treatment 7,Treatment 8,Treatment 8,Treatment 8,Treatment 8,Treatment 8,Treatment 8,Treatment 8,Treatment 8,Treatment 8,Treatment 9,Treatment 9,Treatment 9,Treatment 9,Treatment 9,Treatment 9,Treatment 9,Treatment 9,Treatment 9,Treatment 10,Treatment 10,Treatment 11,Treatment 11,Treatment 12,Treatment 12,Treatment 13,Treatment 13,Treatment 14,Treatment 14,Treatment 15,Treatment 15,Treatment 16,Treatment 16,Treatment 17,Treatment 17,Treatment 18,Treatment 18,Treatment 19,Treatment 19,Treatment 20,Treatment 20,Treatment 21,Treatment 21
Treatment,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Treatment 1,1,29,84,127,28,40,39,14,54,55,161,182,12,61,5,9,30,76,99,17,32,86,21,18,11,4,112,36,10,264,106,218,89,33,79,154,62,72,243,31,37,116,8,38,102,25,216,

In [197]:
# Initialize empty lists to store results
index_list = []
column_count_list = []
average_list = []
total_sum_list = []
ideal_rank_list = []

# Iterate through each row
for index, row in cosine_rank.iterrows():
    # Get the index for the current row
    current_index = index
    
    # Filter values in the row that have a column index matching the current_index
    matching_values = row[current_index]
    
    # Check if there are any matching values
    if not matching_values.empty:
        # Calculate column count, average, and total sum for the matching values
        column_count = len(matching_values)
        average_value = matching_values.mean()
        total_sum = matching_values.sum()
        ideal_rank = sum(range(column_count, 0, -1))
    else:
        # If no matching values, set column count to 0, average to NaN, and total sum to 0
        column_count = 0
        average_value = float('nan')
        total_sum = 0
        ideal_rank = 0
    
    # Append results to lists
    index_list.append(current_index)
    column_count_list.append(column_count)
    average_list.append(average_value)
    total_sum_list.append(total_sum)
    ideal_rank_list.append(ideal_rank)

# Create a new DataFrame with the results
result_df = pd.DataFrame({'Index': index_list, 'ColumnCount': column_count_list, 'Average': average_list, 'TotalSum': total_sum_list, 'IdealRank': ideal_rank_list})

# Calculate the PerformanceMetric outside the for loop
result_df['PerformanceMetric'] = result_df['IdealRank'] / result_df['TotalSum']
# Calculate the average of the 'PerformanceMetric' column
performance_metric = result_df['PerformanceMetric'].mean()

# Display the result DataFrame
result_df, performance_metric

(            Index  ColumnCount     Average  TotalSum  IdealRank  \
 0     Treatment 1          193  136.310881     26308      18721   
 1     Treatment 1          193  132.865285     25643      18721   
 2     Treatment 1          193  131.476684     25375      18721   
 3     Treatment 1          193  136.585492     26361      18721   
 4     Treatment 1          193  132.943005     25658      18721   
 ..            ...          ...         ...       ...        ...   
 287  Treatment 19            2    1.500000         3          3   
 288  Treatment 20            2    3.000000         6          3   
 289  Treatment 20            2    3.500000         7          3   
 290  Treatment 21            2    2.000000         4          3   
 291  Treatment 21            2    4.000000         8          3   
 
      PerformanceMetric  
 0             0.711609  
 1             0.730063  
 2             0.737773  
 3             0.710178  
 4             0.729636  
 ..                 ...  


In [198]:
performance_metric

0.5711472548369091